In [30]:
import requests

url = "https://en.wikipedia.org/wiki/List_of_association_football_stadiums_by_capacity"

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    )
}

resp = requests.get(url, headers=headers)
resp.raise_for_status()   # sẽ raise nếu 4xx/5xx

html = resp.text
print(html)  # in thử 1000 ký tự đầu để confirm


<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" lang="en" dir="ltr">
<head>
<meta charset="UTF-8">
<title>List of association football stadiums by capacity - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limite

In [31]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')
table = soup.select_one("table.wikitable.sortable")

rows = table.find_all('tr')

rows


[<tr>
 <th>Rank
 </th>
 <th>Stadium
 </th>
 <th>Seating capacity
 </th>
 <th>Region
 </th>
 <th>Country
 </th>
 <th>City
 </th>
 <th>Images
 </th>
 <th>Home team(s)
 </th></tr>,
 <tr>
 <th>1
 </th>
 <td style="background-color:#9FC"><b><a href="/wiki/Rungrado_1st_of_May_Stadium" title="Rungrado 1st of May Stadium">Rungrado 1st of May Stadium</a></b> ♦</td>
 <td>114,000<sup class="reference" id="cite_ref-1"><a href="#cite_note-1"><span class="cite-bracket">[</span>1<span class="cite-bracket">]</span></a></sup></td>
 <td>East Asia</td>
 <td><span data-sort-value="North Korea"><span class="flagicon"><span class="mw-image-border" typeof="mw:File"><span><img alt="" class="mw-file-element" data-file-height="600" data-file-width="1200" decoding="async" height="12" src="//upload.wikimedia.org/wikipedia/commons/thumb/5/51/Flag_of_North_Korea.svg/40px-Flag_of_North_Korea.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/5/51/Flag_of_North_Korea.svg/60px-Flag_of_North_Korea.svg.png 

In [32]:
def clean_text(text):
    text = str(text).strip()
    text = text.replace('&nbsp', '')
    if text.find(' ♦'):
        text = text.split(' ♦')[0]
    if text.find('[') != -1:
        text = text.split('[')[0]
    if text.find(' (formerly)') != -1:
        text = text.split(' (formerly)')[0]

    return text.replace('\n', '')

In [35]:
import json


data = []

for i in range(1, len(rows)):
    tds = rows[i].find_all('td')
    values = {
        'rank': i,
        'stadium': clean_text(tds[0].text),
        'capacity': clean_text(tds[1].text).replace(',', '').replace('.', ''),
        'region': clean_text(tds[2].text),
        'country': clean_text(tds[3].text),
        'city': clean_text(tds[4].text),
        'images': 'https://' + tds[5].find('img').get('src').split("//")[1] if tds[5].find('img') else "NO_IMAGE",
        'home_team': clean_text(tds[6].text),
    }
    data.append(values)

json_rows = json.dumps(data)
# kwargs['ti'].xcom_push(key='rows', value=json_rows)



In [38]:
json_rows

'[{"rank": 1, "stadium": "Rungrado 1st of May Stadium", "capacity": "114000", "region": "East Asia", "country": "North Korea", "city": "Pyongyang", "images": "https://upload.wikimedia.org/wikipedia/commons/thumb/8/81/Arirang_Mass_Games_12.JPG/250px-Arirang_Mass_Games_12.JPG", "home_team": "Korea DPR national football team, Korea DPR women\'s national football team, April 25"}, {"rank": 2, "stadium": "Camp Nou", "capacity": "99354", "region": "Europe", "country": "Spain", "city": "Barcelona, Catalonia", "images": "https://upload.wikimedia.org/wikipedia/commons/thumb/0/0e/Camp_Nou_Experience_%28_Ank_Kumar%2C_INFOSYS_Limited_%29_07.jpg/250px-Camp_Nou_Experience_%28_Ank_Kumar%2C_INFOSYS_Limited_%29_07.jpg", "home_team": "FC Barcelona"}, {"rank": 3, "stadium": "FNB Stadium", "capacity": "94736", "region": "Africa", "country": "South Africa", "city": "Johannesburg, Gauteng", "images": "https://upload.wikimedia.org/wikipedia/commons/thumb/c/c0/First_game_of_the_2010_FIFA_World_Cup%2C_South_Af

In [ ]:
geolocator = Nominatim(user_agent='geoapiExercises')
location = geolocator.geocode(f'{city}, {country}')